In [19]:
import requests
import json
import pandas as pd

url = "https://v3.football.api-sports.io/"
endpoints = ["countries"]

headers = {
	'x-rapidapi-host': "v3.football.api-sports.io",
    'x-rapidapi-key': "fb2140228973d644db847895c454c22b"
}

params = {}

def api_req(endpoint, params):
    response = requests.get(
        URL+endpoint,
        headers = headers,
        params = params
    )
    if response.status_code == 200:
            
        remaining = response.headers.get("x-ratelimit-requests-remaining")
        data = response.json()
        print(f"requests before reaching limit {remaining}", data)

    else:
        print(f"Error {response.status_code}, {response.text}")

    return data

#####  Countries

In [20]:
endpoint = 'countries'
params = {}
response = api_req(endpoint,params)

requests before reaching limit 98 {'get': 'countries', 'parameters': [], 'errors': [], 'results': 171, 'paging': {'current': 1, 'total': 1}, 'response': [{'name': 'Albania', 'code': 'AL', 'flag': 'https://media.api-sports.io/flags/al.svg'}, {'name': 'Algeria', 'code': 'DZ', 'flag': 'https://media.api-sports.io/flags/dz.svg'}, {'name': 'Andorra', 'code': 'AD', 'flag': 'https://media.api-sports.io/flags/ad.svg'}, {'name': 'Angola', 'code': 'AO', 'flag': 'https://media.api-sports.io/flags/ao.svg'}, {'name': 'Antigua-And-Barbuda', 'code': 'AG', 'flag': 'https://media.api-sports.io/flags/ag.svg'}, {'name': 'Argentina', 'code': 'AR', 'flag': 'https://media.api-sports.io/flags/ar.svg'}, {'name': 'Armenia', 'code': 'AM', 'flag': 'https://media.api-sports.io/flags/am.svg'}, {'name': 'Aruba', 'code': 'AW', 'flag': 'https://media.api-sports.io/flags/aw.svg'}, {'name': 'Australia', 'code': 'AU', 'flag': 'https://media.api-sports.io/flags/au.svg'}, {'name': 'Austria', 'code': 'AT', 'flag': 'https:/

In [61]:
countries = pd.DataFrame(columns=['name', 'code'])
for country in response['response']:
    countries.loc[len(countries)] = {'name': country['name'], 'code': country['code']}
countries.head()

,name,code
0,Albania,AL
1,Algeria,DZ
2,Andorra,AD
3,Angola,AO
4,Antigua-And-Barbuda,AG


In [62]:
countries.to_csv("countries.csv")

#### Leagues

##### Code

In [ ]:
endpoint = 'leagues'
params = {}
leagues = api_req(endpoint, params)

In [67]:
def get_cols(response, coverage):
    coverages = response
    for key, val in coverages.items():
        if isinstance(val, dict):
            coverage = get_cols(val, coverage)
        else:
            coverage.append(key)
    return coverage

In [70]:
covs = []
coverages = get_cols(leagues['response'][0]['seasons'][0], covs)

In [186]:
# functions inserting data into contests and seasons dfs
def insert_contests(response):
    contests_rows = []
    for league in response['response']:
        contest_row = {'league_id': league['league']['id'],
                       'name': league['league']['name'],
                       'type': league['league']['type'],
                       'country': league['country']['name'],
                       'country_code': league['country']['code'],
                       'seasons': len(league['seasons'])}
        contests_rows.append(contest_row)
    contests = pd.DataFrame(contests_rows)
    return contests

def insert_seasons(response):
    season_rows = []
    for league in response['response']:
        for season in league['seasons']:
            row = {}
            row['league_id'] = league['league']['id']
            for key, val in season.items():

                if key == 'coverage':
                    for k, v in season['coverage'].items():
                        if k == 'fixtures':
                            for a, b in season['coverage']['fixtures'].items():
                                row[a] = b
                        else:
                            row[k] = v
                else:
                    row[key] = val
            season_rows.append(row)
    
    seasons = pd.DataFrame(season_rows)
    return seasons

In [187]:
contests = insert_contests(leagues)
seasons = insert_seasons(leagues)

In [188]:
contests.to_csv('contests.csv')
seasons.to_csv('seasons.csv')

##### Df information

In [189]:
contests.head()

,league_id,name,type,country,country_code,seasons
0,4,Euro Championship,Cup,World,None,5
1,21,Confederations Cup,Cup,World,None,3
2,61,Ligue 1,League,France,FR,14
3,144,Jupiler Pro League,League,Belgium,BE,14
4,71,Serie A,League,Brazil,BR,15


In [190]:
seasons.head()

,league_id,year,start,end,current,events,lineups,statistics_fixtures,statistics_players,standings,players,top_scorers,top_assists,top_cards,injuries,predictions,odds
0,4,2008,2008-06-07,2008-06-29,False,True,True,False,False,False,True,True,True,True,False,True,False
1,4,2012,2012-06-08,2012-07-01,False,True,True,False,False,False,True,True,True,True,False,True,False
2,4,2016,2016-06-10,2016-07-10,False,True,True,True,True,True,True,True,True,True,False,True,False
3,4,2020,2019-03-21,2021-07-11,False,True,True,True,True,True,True,True,True,True,False,True,False
4,4,2024,2024-06-14,2024-06-26,True,False,False,False,False,True,False,False,False,False,False,True,False


In [161]:
print(f'contests size: {contests.size}', f'seasons size: {seasons.size}')

contests size: 6648 seasons size: 98992


In [162]:
contests.describe()

,league_id,seasons
count,1108.000000,1108.000000
mean,554.500000,5.583935
std,319.996354,3.787155
min,1.000000,1.000000
25%,277.750000,3.000000
50%,554.500000,5.000000
75%,831.250000,8.000000
max,1108.000000,15.000000


In [163]:
seasons.describe()

,year
count,6187.000000
mean,2019.561985
std,3.332549
min,2008.000000
25%,2018.000000
50%,2020.000000
75%,2022.000000
max,2026.000000


In [167]:
contests.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1108 entries, 0 to 1107
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   league_id     1108 non-null   int64 
 1   name          1108 non-null   object
 2   type          1108 non-null   object
 3   country       1108 non-null   object
 4   country_code  958 non-null    object
 5   seasons       1108 non-null   int64 
dtypes: int64(2), object(4)
memory usage: 52.1+ KB


In [166]:
seasons.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6187 entries, 0 to 6186
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   year                 6187 non-null   int64 
 1   start                6187 non-null   object
 2   end                  6187 non-null   object
 3   current              6187 non-null   bool  
 4   events               6187 non-null   bool  
 5   lineups              6187 non-null   bool  
 6   statistics_fixtures  6187 non-null   bool  
 7   statistics_players   6187 non-null   bool  
 8   standings            6187 non-null   bool  
 9   players              6187 non-null   bool  
 10  top_scorers          6187 non-null   bool  
 11  top_assists          6187 non-null   bool  
 12  top_cards            6187 non-null   bool  
 13  injuries             6187 non-null   bool  
 14  predictions          6187 non-null   bool  
 15  odds                 6187 non-null   bool  
dtypes: boo

#### Leagues number

In [169]:
print(len(contests['league_id'].unique()))

1108


In [176]:
leagues_per_c = contests.groupby('country')['league_id'].count().reset_index()

In [177]:
leagues_per_c.sort_values(by='league_id', ascending=False).head(30)

,country,league_id
167,World,150
23,Brazil,72
48,England,45
144,Spain,38
59,Germany,33
78,Italy,29
15,Belgium,28
55,France,24
8,Australia,22
128,Portugal,19


In [178]:
contests[contests['country']=='Poland'].head(11)

,league_id,name,type,country,country_code,seasons
357,106,Ekstraklasa,League,Poland,PL,6
371,107,I Liga,League,Poland,PL,6
464,109,II Liga - East,League,Poland,PL,5
561,108,Cup,Cup,Poland,PL,5
672,780,III Liga - Group 1,League,Poland,PL,4
673,781,III Liga - Group 2,League,Poland,PL,4
674,782,III Liga - Group 3,League,Poland,PL,4
675,783,III Liga - Group 4,League,Poland,PL,4
770,676,Central Youth League,League,Poland,PL,4
815,727,Super Cup,Cup,Poland,PL,4


In [191]:
seasons[seasons['injuries']==True].info()

<class 'pandas.core.frame.DataFrame'>
Index: 65 entries, 18 to 2978
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   league_id            65 non-null     int64 
 1   year                 65 non-null     int64 
 2   start                65 non-null     object
 3   end                  65 non-null     object
 4   current              65 non-null     bool  
 5   events               65 non-null     bool  
 6   lineups              65 non-null     bool  
 7   statistics_fixtures  65 non-null     bool  
 8   statistics_players   65 non-null     bool  
 9   standings            65 non-null     bool  
 10  players              65 non-null     bool  
 11  top_scorers          65 non-null     bool  
 12  top_assists          65 non-null     bool  
 13  top_cards            65 non-null     bool  
 14  injuries             65 non-null     bool  
 15  predictions          65 non-null     bool  
 16  odds        

#### Team per country

In [195]:
contests[contests['country']!='World'].sort_values(by='league_id').head(5)

,league_id,name,type,country,country_code,seasons
5,39,Premier League,League,England,GB,14
110,40,Championship,League,England,GB,13
116,41,League One,League,England,GB,13
117,42,League Two,League,England,GB,13
115,43,National League,League,England,GB,13


In [197]:
endpoint = 'teams'
params = {'league': 39}
teams_per_c = api_req(endpoint, params)

requests before reaching limit 94 {'get': 'teams', 'parameters': {'league': '39'}, 'errors': {'season': 'The Season field is required.'}, 'results': 0, 'paging': {'current': 1, 'total': 1}, 'response': []}
